Инициализация рабочей директории и путей

In [1]:
import os
import sys
from pathlib import Path

def init_work_dir(path):
    # Установить рабочую директорию
    project_root = Path(path)
    os.chdir(project_root)
    # Добавить путь к модулям проекта
    sys.path.insert(0, str(project_root))
    print(f"Текущая рабочая директория: {os.getcwd()}")

init_work_dir("C:/Projects/GeoOffice")

DB_PATH = r'C:\GeoOffice_test_server\projects.db'

Текущая рабочая директория: C:\Projects\GeoOffice


Установка / снятие защиты

In [11]:
import subprocess

def show_db_file():
    subprocess.run(['attrib', '-h', '-s', str(DB_PATH)], check=True)
def protect_db_file():
    subprocess.run(['attrib', '+h', '+s', str(DB_PATH)], check=True)

# show_db_file()  # Снятие защиты
# protect_db_file()  # Установка защиты

Удаление БД

In [2]:
from pathlib import Path

def delete_db_file():
    Path(DB_PATH).unlink()
    if Path(DB_PATH).exists():
        raise Exception("Файл не удалён")

# delete_db_file()

Создание / Подключение к БД

In [2]:
from src.services.database_service import DataBaseProjectService

db_service = DataBaseProjectService(DB_PATH)

20:41:19 | INFO     | GeoOffice | ============================================================
20:41:19 | INFO     | GeoOffice | 🚀 Запуск приложения GeoOffice
20:41:19 | INFO     | GeoOffice | 📅 Дата и время: 2025-07-05 20:41:19
20:41:19 | INFO     | GeoOffice | 🐍 Версия Python: 3.13.5 (tags/v3.13.5:6cb20a2, Jun 11 2025, 16:15:46) [MSC v.1943 64 bit (AMD64)]
20:41:19 | INFO     | GeoOffice | 📁 Рабочая директория: C:\Projects\GeoOffice
20:41:19 | INFO     | GeoOffice | 📂 Директория логов: C:\Projects\GeoOffice\storage\logs
20:41:19 | INFO     | GeoOffice | ============================================================
20:41:19 | INFO     | GeoOffice.services.database_service | Инициализация базы данных: C:\GeoOffice_test_server\projects.db
20:41:19 | INFO     | GeoOffice.services.database_service | База данных успешно инициализирована


Заполнение БД тестовыми данными

In [4]:
test_projects = [
    {
        'number': '50.25',
        'name': 'Реконструкция спортзала по адресу ул.Молодежная 26 в г.Новополоцк',
        'customer': 'Новополоцкий городской исполнительный комитет',
        'chief_engineer': 'Богданова Екатерина Александровна',
        'status': 'активный',
        'address': 'ул.Молодежная 26 в г.Новополоцк',
        'path': '2025\\Новополоцк\\50.25 Реконструкция спортзала Новополоцк'
    },
    {
        'number': '116.25',
        'name': 'Ушачи вынос осей',
        'customer': 'Ушачский УКС',
        'chief_engineer': 'Кундро Нина Викторовна',
        'status': 'активный',
        'address': 'гп.Ушачи ул.Проектируемая 1',
        'path': '2025\\Ушачский район\\116.25 Ушачи вынос осей'
    },
    {
        'number': '72.25',
        'name': 'ЖД Шарковщина',
        'customer': 'ОАО "Княж"',
        'chief_engineer': 'Понятаева Наталья Николаевна',
        'status': 'активный',
        'address': 'г.Шарковщина',
        'path': '2025\\Шарковщинский район\\72.25 ЖД Шарковщина'
    },
    ]

In [5]:
for test_project in test_projects:
    db_service.create_project(**test_project)

20:30:22 | DEBUG    | GeoOffice.services.database_service | Создан новый проект: Project(id=None, number='50.25', name='Реконструкция спортзала по адресу ул.Молодежная 26 в г.Новополоцк')
20:30:22 | DEBUG    | GeoOffice.services.database_service | Создан новый проект: Project(id=None, number='116.25', name='Ушачи вынос осей')
20:30:22 | DEBUG    | GeoOffice.services.database_service | Создан новый проект: Project(id=None, number='72.25', name='ЖД Шарковщина')


In [22]:
from datetime import datetime
from src.models.project_model import Project

project_data = db_service.get_project(1).to_dict()
project_data['created_date'] = datetime.fromisoformat(project_data['created_date'])
project_data['modified_date'] = datetime.fromisoformat(project_data['modified_date'])
Project(**project_data)

20:59:19 | DEBUG    | GeoOffice.services.database_service | Получение проекта по id: id=1


Project(id=1, number='50.25', name='Реконструкция спортзала по адресу ул.Молодежная 26 в г.Новополоцк', customer='Новополоцкий городской исполнительный комитет', chief_engineer='Богданова Екатерина Александровна', status='активный', address='ул.Молодежная 26 в г.Новополоцк', path='2025\\Новополоцк\\50.25 Реконструкция спортзала Новополоцк', created_date=datetime.datetime(2025, 7, 5, 20, 30, 22, 952641), modified_date=datetime.datetime(2025, 7, 5, 20, 30, 22, 952649))

In [20]:
db_service.models.Project.select()[:]

[ProjectTable[1], ProjectTable[2], ProjectTable[3]]